# Experiments with Wandb sweep

I experimented with the Wandb sweep. However, at the end I was not satisfied with the combinations of hyperparameters that were found, so I chose epochs manually (by analying evaluation during training).

Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
!pip install -q transformers

In [3]:
# Install the simpletransformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [4]:
# Install wandb
!pip install -q wandb

In [5]:
import wandb

In [6]:
# Login to wandb
wandb.login()

In [7]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [8]:
# FTD
train_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-train.txt", sep="\t", index_col=0)
dev_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-dev.txt", sep = "\t", index_col = 0)
test_df = pd.read_csv("/kaggle/input/genredatasetscomparison/FTD-test.txt", sep = "\t", index_col = 0)

print("FTD train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

In [9]:
train_df.head()

## Training and testing

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [10]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("FTD-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [11]:
# Open the main results file:

previous_results_file = open("./FTD-Experiments-Results.json")
previous_results = json.load(previous_results_file)
len(previous_results)

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [13]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

## Hyperparameter sweeps

In [ ]:
# Configure the WandB sweep for hyperparameter search
sweep_config = {
    "method": "grid",  # random, grid, bayes
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [10, 20, 30, 50, 70, 90]},
    },
}

In [ ]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="FTD-learning-hyperparameter-sweep")

In [ ]:
# Add logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# Create a function that will be ran inside the sweep
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "labels_list": LABELS,
            "no_cache": True,
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            "evaluate_during_training":True,
            'logging_steps': 10,
            'evaluate_during_training_steps': 10,
            "use_cached_eval_features": True,
            "reprocess_input_data": True,
            "silent": True,
            "wandb_project": 'FTD-learning-hyperparameter-sweep',
            "sweep_config":wandb.config
            }
        )

    # Train the model
    roberta_base_model.train_model(train_df, eval_df=dev_df)

    # Evaluate the model
    roberta_base_model.eval_model(dev_df)

    # Sync wandb
    wandb.join()

In [ ]:
# Run the sweeps
wandb.agent(sweep_id, train, count= 10)